In [1]:
import gym, random, tempfile
import numpy as np
from gym import wrappers
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:

gamma=0.99
epsilon_decay = 0.003
learning_rate =0.0001
max_e=1
min_e=0.0001
replay_batch_size=30
iterations=10000

In [ ]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            ### sample from memory and train online network
            # a sample: [state,action,reward,new_state,done]
            batch=np.array(random.sample(memory,replay_batch_size))
            # X: current state
            X=np.vstack(batch[:,0])
            # default Y-target: q(X) of online model
            Y_target=online_model.predict(X)
            non_terminal_samples = np.where(batch[:, 4] == False)
            #target = rewards for terminal state
            target= np.array([b[2] for b in batch])
            #target = rewards +new qvalue for non-terminal state
            if batch[non_terminal_samples].shape[0]>0:
                            # Qvalues for new state, 2 network
                online_qvalues =online_model.predict(np.vstack(batch[:, 3]))
                target_qvalues = target_model.predict(np.vstack(batch[:, 3]))
                target[non_terminal_samples] = target[non_terminal_samples]+\
                gamma * target_qvalues[non_terminal_samples,np.argmax(online_qvalues[non_terminal_samples, :][0], axis=1)][0]       
            actions = np.array([int(b[1]) for b in batch])
            Y_target[:, actions] = target
            online_model.fit(np.array(X), np.array(Y_target), verbose=0)
#             Y_target=[[0,0,0,0] for _ in range(replay_batch_size)]
#             X=[[0,0,0,0] for _ in range(replay_batch_size)]
#             # i:index for sample [state,action,reward,new_state,done]
#             for i,sample in enumerate(batch):
#                 terminal_sample = sample[4]
#                 new_state_sample = sample[3]
#                 state_sample = sample[0]
#                 X[i]=state_sample
#                 action_sample = sample[1]
#                 Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
#                 if terminal:
#                     #terminal then targetY is reward
#                     Y_target[i][action_sample] = sample[2]
#                 else:
#                     #use online weights to select argmax action under new_state

#                     online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
#                     max_action = np.argmax(online_qvalues_batch)
#                     #use target weights to calculate Qvalue
#                     #non-terminal then targetY is reward +gamma*new Qvalue
#                     Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

#             online_model.fit(np.array(X), np.array(Y_target), verbose=0)

            if done:
                target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward)  
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -292.43  rolling reward:  -292.43  Steps:  114  epsilon:  1.00


In [165]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 